In [1]:
# Convert the 'Symbol' column to a list
symbol_list1 = ['SPY', 'QQQ','IWM','DIA','SOXX','SLV','GLD','XLV','XLU','XLRE','XLP','XLE','XLB','XLY','XLF','XLC','XLI','XLK','TLT','MAGS']
symbol_list2 = (['TRV','XYZ']+['AMD', 'NVDA','LRCX']+
                   ['UNH','CI','MRK']+
                   ['HAS']+
                   ['BKR','MPLX']+
                   ['PAAS','SVM','NGD','NEM','SCCO','TGB','DD','CTVA']+
                   ['D','NEE']+
                   ['UBER']+
                   ['NVO'])
symbol_list3= (['NVDA']+['META', 'AMZN','GOOGL']+                 
                   ['AAPL','TSLA','MSFT'])# Replace with your desired symbols
symbol_list=symbol_list1+symbol_list2+symbol_list3
# 使用 set 去重
symbol_list = list(set(symbol_list))
# Display the result
print("Symbols as a list:", symbol_list)


Symbols as a list: ['MRK', 'SVM', 'XLI', 'BKR', 'UBER', 'XLV', 'HAS', 'SPY', 'XLC', 'DIA', 'NGD', 'TSLA', 'TLT', 'IWM', 'GLD', 'AMD', 'UNH', 'AMZN', 'TGB', 'NVO', 'XLP', 'XLRE', 'XYZ', 'AAPL', 'NEM', 'XLE', 'META', 'GOOGL', 'NVDA', 'PAAS', 'CTVA', 'QQQ', 'NEE', 'XLK', 'XLY', 'TRV', 'XLF', 'MSFT', 'LRCX', 'SCCO', 'XLB', 'SLV', 'DD', 'MAGS', 'SOXX', 'MPLX', 'XLU', 'CI', 'D']


In [2]:
import requests
%run constant.ipynb
def get_historical_data(symbol, access_token, period_type='day', period=1, frequency_type='daily', frequency=1, need_extended_hours_data=False, need_previous_close=True):
    base_url = 'https://api.schwabapi.com/marketdata/v1/pricehistory'
    headers = {
        'accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        'symbol': symbol,
        'periodType': "year",
        'period': 1,
        'frequencyType': "daily",
        'frequency': 1,       
    }
    
    response = requests.get(base_url, headers=headers, params=params)
    return response.json()

# Example usage
access_token = get_new_tokens()
symbol = 'SPY'
historical_data = get_historical_data(symbol, access_token)
print (historical_data['candles'][0:10])


[{'open': 497.72, 'high': 498.41, 'low': 494.45, 'close': 496.76, 'volume': 71736740, 'datetime': 1708408800000}, {'open': 495.42, 'high': 497.37, 'low': 493.56, 'close': 497.21, 'volume': 59603771, 'datetime': 1708495200000}, {'open': 504.01, 'high': 508.49, 'low': 503.02, 'close': 507.5, 'volume': 76402535, 'datetime': 1708581600000}, {'open': 509.27, 'high': 510.13, 'low': 507.1, 'close': 507.85, 'volume': 61321818, 'datetime': 1708668000000}, {'open': 508.3, 'high': 508.75, 'low': 505.86, 'close': 505.99, 'volume': 50386738, 'datetime': 1708927200000}, {'open': 506.7, 'high': 507.16, 'low': 504.75, 'close': 506.93, 'volume': 48854528, 'datetime': 1709013600000}, {'open': 505.33, 'high': 506.855, 'low': 504.96, 'close': 506.26, 'volume': 56506634, 'datetime': 1709100000000}, {'open': 508.07, 'high': 509.74, 'low': 505.35, 'close': 508.08, 'volume': 83924795, 'datetime': 1709186400000}, {'open': 508.98, 'high': 513.29, 'low': 508.56, 'close': 512.85, 'volume': 76844844, 'datetime': 1

In [3]:
from datetime import datetime, timedelta

def is_friday_night_to_monday_morning():
    now = datetime.now()
    
    # 获取当前星期几 (0是周一，6是周日)
    current_weekday = now.weekday()
    
    # 获取当前时间
    current_time = now.time()
    
    # 定义星期五晚上7点和星期一早上4点的时间
    friday_night = datetime.strptime("19:00:00", "%H:%M:%S").time()
    monday_morning = datetime.strptime("04:00:00", "%H:%M:%S").time()
    
    # 判断当前时间是否在星期五晚上7点到星期一早上4点之间
    if current_weekday == 4 and current_time >= friday_night:  # 星期五晚上7点之后
        return True
    elif current_weekday == 0 and current_time < monday_morning:  # 星期一早上4点之前
        return True
    elif current_weekday == 5 or current_weekday == 6:  # 星期六或星期日
        return True
    else:
        return False

# 测试函数
if is_friday_night_to_monday_morning():
    print("当前时间在星期五晚上7点到星期一早上4点之间。")
else:
    print("当前时间不在星期五晚上7点到星期一早上4点之间。")
weekend=is_friday_night_to_monday_morning()

当前时间不在星期五晚上7点到星期一早上4点之间。


In [4]:
import requests
from datetime import datetime
import pytz  # For timezone conversion
%run constant.ipynb
# Set up Houston timezone
houston_tz = pytz.timezone('America/Chicago')

def combine_data(current_price, access_token):
    combined_data = []  # 最终结果列表
    weekend=is_friday_night_to_monday_morning()
    for item in current_price:
        symbol = item['symbol']
        print(f"Processing symbol: {symbol}")
        
        # 使用API获取历史数据
        historical_data = get_historical_data(symbol, access_token)
        historical_data = historical_data['candles']  # 提取历史数据中的candles列表
        
        # 将当前数据（item）附加到历史数据中
        combined_entry = historical_data.copy()  # 复制历史数据，避免修改原始数据
        if weekend==False:
           combined_entry.append(item)  # 将当前数据添加到历史数据中
           # 创建新的字典，键是symbol，值是合并后的数据
           new_data = {symbol: combined_entry}
        else:
           new_data = {symbol:historical_data}
        
        # 将新字典添加到最终结果列表中
        combined_data.append(new_data)
    
    return combined_data

# Fetch current data for a single symbol
def get_current_data(symbol, access_token):
    base_url = f'https://api.schwabapi.com/marketdata/v1/{symbol}/quotes'
    headers = {
        'accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    
    response = requests.get(base_url, headers=headers)
    response.raise_for_status()  # Raise an error for bad responses
    data = response.json()
    
    #print (data)
    # Extract the needed data from the response
    symbol_data = data.get(symbol, {})
    last_price = symbol_data.get('regular', {}).get('regularMarketLastPrice', None)
    trade_time = symbol_data.get('regular', {}).get('regularMarketTradeTime', None)
    high_price = symbol_data.get('quote', {}).get('highPrice', None)
    low_price = symbol_data.get('quote', {}).get('lowPrice', None)
    open_price = symbol_data.get('quote', {}).get('openPrice', None)
    close_price = symbol_data.get('quote', {}).get('closePrice', None)   

    # Return a dictionary with the symbol, last price, and trade time
    return {'symbol': symbol, 'last': last_price,'high':high_price,'open':open_price,'close':close_price,'low':low_price,'datetime': trade_time}

# Fetch current data for a list of symbols
def get_current_data_for_symbols(symbols, access_token):
    results = []
    for symbol in symbols:
        # Fetch data for each symbol
        symbol_data = get_current_data(symbol, access_token)
        # Append the result to the list
        results.append(symbol_data)
    return results

# Example usage
access_token = get_new_tokens()  # Assuming you already have a function to get the access token
symbols = symbol_list  # List of symbols
current_price=[]
all_data=[]
#这一步不耗费时间，所以直接用了
current_price = get_current_data_for_symbols(symbols, access_token)  
all_data_daily=combine_data(current_price, access_token)  


Processing symbol: MRK
Processing symbol: SVM
Processing symbol: XLI
Processing symbol: BKR
Processing symbol: UBER
Processing symbol: XLV
Processing symbol: HAS
Processing symbol: SPY
Processing symbol: XLC
Processing symbol: DIA
Processing symbol: NGD
Processing symbol: TSLA
Processing symbol: TLT
Processing symbol: IWM
Processing symbol: GLD
Processing symbol: AMD
Processing symbol: UNH
Processing symbol: AMZN
Processing symbol: TGB
Processing symbol: NVO
Processing symbol: XLP
Processing symbol: XLRE
Processing symbol: XYZ
Processing symbol: AAPL
Processing symbol: NEM
Processing symbol: XLE
Processing symbol: META
Processing symbol: GOOGL
Processing symbol: NVDA
Processing symbol: PAAS
Processing symbol: CTVA
Processing symbol: QQQ
Processing symbol: NEE
Processing symbol: XLK
Processing symbol: XLY
Processing symbol: TRV
Processing symbol: XLF
Processing symbol: MSFT
Processing symbol: LRCX
Processing symbol: SCCO
Processing symbol: XLB
Processing symbol: SLV
Processing symbol: D

In [5]:
def calculate_ma(all_data_daily):
    ma_results = []  # 存储最终结果的列表

    for symbol_data in all_data_daily:
        for symbol, data in symbol_data.items():
            # 提取收盘价（close）数据
            close_prices = [day['close'] for day in data if 'close' in day]
            
            if len(close_prices) == 0:
                # 没有足够的数据
                ma_results.append({symbol: {'ma5': None, 'ma20': None}})
                continue
            
            last_close = close_prices[-1]  # 最后一天的收盘价

            # 计算 MA5
            if len(close_prices) >= 5:  # 确保有足够的数据点
                ma5 = sum(close_prices[-5:]) / 5  # 取最后5天的收盘价计算平均值
            else:
                ma5 = None  # 如果数据不足，返回None

            # 计算 MA20
            if len(close_prices) >= 20:  # 确保有足够的数据点
                ma20 = sum(close_prices[-20:]) / 20  # 取最后20天的收盘价计算平均值
            else:
                ma20 = None  # 如果数据不足，返回None

            if ma5 is not None and ma20 is not None:
                if last_close < ma5 and ma5 < ma20:
                    ma5, ma20 = -100, -100
                elif last_close > ma5 and ma5 > ma20:
                    ma5, ma20 = 100, 100
                else:
                    ma5 = 100 * (last_close - ma5) / ma5 if ma5 != 0 else None
                    ma20 = 100 * (last_close - ma20) / ma20 if ma20 != 0 else None

            # 将结果添加到ma_results中
            ma_results.append({symbol: {'ma5': ma5, 'ma20': ma20}})
    
    return ma_results


ma_results = calculate_ma(all_data_daily)
print(ma_results)


[{'MRK': {'ma5': 1.4410314751611608, 'ma20': -5.917523960256764}}, {'SVM': {'ma5': 100, 'ma20': 100}}, {'XLI': {'ma5': 0.3238819565940396, 'ma20': 0.2539400441415098}}, {'BKR': {'ma5': 100, 'ma20': 100}}, {'UBER': {'ma5': 100, 'ma20': 100}}, {'XLV': {'ma5': 0.4790599991764063, 'ma20': 0.08648183027000307}}, {'HAS': {'ma5': 100, 'ma20': 100}}, {'SPY': {'ma5': 100, 'ma20': 100}}, {'XLC': {'ma5': 100, 'ma20': 100}}, {'DIA': {'ma5': 100, 'ma20': 100}}, {'NGD': {'ma5': -100, 'ma20': -100}}, {'TSLA': {'ma5': 0.8835988606667046, 'ma20': -3.8625877057911575}}, {'TLT': {'ma5': -0.29388493274556926, 'ma20': -0.17032497552640385}}, {'IWM': {'ma5': -100, 'ma20': -100}}, {'GLD': {'ma5': 100, 'ma20': 100}}, {'AMD': {'ma5': 0.8582936138030616, 'ma20': 0.05277873496786643}}, {'UNH': {'ma5': -100, 'ma20': -100}}, {'AMZN': {'ma5': -100, 'ma20': -100}}, {'TGB': {'ma5': -3.2863849765258144, 'ma20': 2.1825396825396846}}, {'NVO': {'ma5': 2.752384003154023, 'ma20': -0.44465665940493865}}, {'XLP': {'ma5': 100

In [6]:
def calculate_kdj(all_data_daily, n=9):
    kdj_results = []  # 存储最终结果的列表

    for symbol_data in all_data_daily:
        for symbol, data in symbol_data.items():
            # 提取收盘价、最高价、最低价
            close_prices = [day['close'] for day in data if 'close' in day]
            high_prices = [day['high'] for day in data if 'high' in day]
            low_prices = [day['low'] for day in data if 'low' in day]

            # 初始化 K、D、J 值
            k_values = []
            d_values = []
            j_values = []
            kd_list = []
            j_count = 0  # 用于记录 J 值连续大于 100 或小于 0 的天数

            for i in range(len(close_prices)):
                if i < n - 1:  # 前 n-1 天数据不足，无法计算 KDJ
                    k_values.append(None)
                    d_values.append(None)
                    j_values.append(None)
                    kd_list.append(None)
                    continue

                # 计算 RSV
                highest_high = max(high_prices[i - n + 1:i + 1])
                lowest_low = min(low_prices[i - n + 1:i + 1])
                rsv = ((close_prices[i] - lowest_low) / (highest_high - lowest_low)) * 100 if (highest_high - lowest_low) != 0 else 50

                # 计算 K 值
                if i == n - 1:  # 第一天计算 KDJ
                    k = rsv
                    d = rsv
                else:
                    k = (2 / 3) * k_values[-1] + (1 / 3) * rsv
                    d = (2 / 3) * d_values[-1] + (1 / 3) * k

                # 计算 J 值
                j = 3 * k - 2 * d

                # 保存 K、D、J 值
                k_values.append(k)
                d_values.append(d)
                j_values.append(j)

                # 计算 kd 值（只保留最后一天的判断结果）
                if i == len(close_prices) - 1:  # 最后一天
                    if d > 80:
                        kd = 100                    
                    elif d < 20:
                        kd = -100                
                    else:
                        kd = k-d  # 其他情况
                else:
                    kd = k-d  # 非最后一天不计算 kd
                kd = round(kd,2)
                # 计算 j 值
                if j > 100:
                    j_count += 1
                    j_value = j_count
                elif j < 0:
                    j_count -= 1
                    j_value = j_count
                else:
                    j_count = 0  # 重置计数器
                    j_value = 0

                # 保存 kd 和 j 值
                kd_list.append({'kd': kd, 'j': j_value})

            # 只保留最后一天的 kd 值
            last_kd = kd_list[-1]['kd'] if kd_list else None
            last_j = kd_list[-1]['j'] if kd_list else None

            # 将结果添加到 kdj_results 中
            kdj_results.append({symbol: {'kd': last_kd, 'j': last_j}})
    
    return kdj_results

# 示例使用
kdj_results = calculate_kdj(all_data_daily)
print(kdj_results)

[{'MRK': {'kd': 13.99, 'j': 0}}, {'SVM': {'kd': -0.63, 'j': 0}}, {'XLI': {'kd': 9.05, 'j': 0}}, {'BKR': {'kd': 1.7, 'j': 0}}, {'UBER': {'kd': 100, 'j': 0}}, {'XLV': {'kd': 7.28, 'j': 0}}, {'HAS': {'kd': 100, 'j': 0}}, {'SPY': {'kd': 100, 'j': 3}}, {'XLC': {'kd': 100, 'j': 0}}, {'DIA': {'kd': 4.22, 'j': 0}}, {'NGD': {'kd': -10.02, 'j': 0}}, {'TSLA': {'kd': 12.93, 'j': 0}}, {'TLT': {'kd': -2.75, 'j': 0}}, {'IWM': {'kd': 1.74, 'j': 0}}, {'GLD': {'kd': 100, 'j': 0}}, {'AMD': {'kd': 14.26, 'j': 0}}, {'UNH': {'kd': -1.68, 'j': 0}}, {'AMZN': {'kd': -0.4, 'j': 0}}, {'TGB': {'kd': -18.25, 'j': 0}}, {'NVO': {'kd': 6.17, 'j': 0}}, {'XLP': {'kd': 100, 'j': 1}}, {'XLRE': {'kd': 100, 'j': 0}}, {'XYZ': {'kd': 4.26, 'j': 0}}, {'AAPL': {'kd': 11.95, 'j': 4}}, {'NEM': {'kd': 100, 'j': 0}}, {'XLE': {'kd': 7.28, 'j': 0}}, {'META': {'kd': -21.05, 'j': 0}}, {'GOOGL': {'kd': 4.43, 'j': 0}}, {'NVDA': {'kd': 100, 'j': 0}}, {'PAAS': {'kd': -17.91, 'j': -1}}, {'CTVA': {'kd': 6.49, 'j': 0}}, {'QQQ': {'kd': 100, '

In [7]:
def calculate_macd(all_data_daily):
    macd_results = []  # 存储最终结果的列表

    for symbol_data in all_data_daily:
        for symbol, data in symbol_data.items():
            # 提取收盘价（close）数据
            close_prices = [day['close'] for day in data if 'close' in day]

            # 计算 EMA12 和 EMA26
            ema12 = calculate_ema(close_prices, 12)
            ema26 = calculate_ema(close_prices, 26)

            # 计算 MACD
            macd = [ema12[i] - ema26[i] for i in range(len(close_prices))]

            # 计算 Signal Line（MACD 的 9 日 EMA）
            signal_line = calculate_ema(macd, 9)

            # 初始化变量
            macd_info = None
            golden_cross_days = 0  # 金叉后的天数
            dead_cross_days = 0    # 死叉后的天数

            for i in range(1, len(macd)):
                # 判断金叉
                if macd[i] > signal_line[i] and macd[i - 1] <= signal_line[i - 1]:  # 金叉条件
                    golden_cross_days = 1
                    dead_cross_days = 0  # 重置死叉天数
                elif macd[i] > signal_line[i]:  # 金叉后持续
                    golden_cross_days += 1
                    dead_cross_days = 0  # 重置死叉天数

                # 判断死叉
                if macd[i] < signal_line[i] and macd[i - 1] >= signal_line[i - 1]:  # 死叉条件
                    dead_cross_days = 1
                    golden_cross_days = 0  # 重置金叉天数
                elif macd[i] < signal_line[i]:  # 死叉后持续
                    dead_cross_days += 1
                    golden_cross_days = 0  # 重置金叉天数

                # 根据规则设置 macd_info
                if macd[i] < 0:
                    if dead_cross_days > 0:  # 死叉
                        macd_info = -dead_cross_days
                    elif golden_cross_days > 0:  # 金叉
                        macd_info = macd[i]  # 正数的 MACD 值
                elif macd[i] > 0:
                    if golden_cross_days > 0:  # 金叉
                        macd_info = golden_cross_days
                    elif dead_cross_days > 0:  # 死叉
                        macd_info = -macd[i]  # 负数的 MACD 值

            # 将结果添加到 macd_results 中
            macd_results.append({symbol: macd_info})
    
    return macd_results

# 计算 EMA 的辅助函数
def calculate_ema(prices, period):
    ema = []
    multiplier = 2 / (period + 1)

    for i in range(len(prices)):
        if i == 0:
            ema.append(prices[i])  # 第一天的 EMA 是当天的价格
        else:
            ema.append((prices[i] - ema[-1]) * multiplier + ema[-1])
    
    return ema

# 示例使用
macd_results = calculate_macd(all_data_daily)
print(macd_results)

[{'MRK': -12}, {'SVM': 48}, {'XLI': -0.4569001658107936}, {'BKR': -0.7532343556821957}, {'UBER': 10}, {'XLV': -0.8382925860155126}, {'HAS': 30}, {'SPY': 23}, {'XLC': 21}, {'DIA': -2.5633562528067273}, {'NGD': -0.04293065148421604}, {'TSLA': -39}, {'TLT': -0.06702460623576201}, {'IWM': -0.03238186000206156}, {'GLD': 33}, {'AMD': -2.3484531410678215}, {'UNH': -8}, {'AMZN': -0.08262346933861409}, {'TGB': 10}, {'NVO': -1.6996322703558349}, {'XLP': 22}, {'XLRE': 25}, {'XYZ': -10}, {'AAPL': 16}, {'NEM': 33}, {'XLE': 3}, {'META': -23.087700396691957}, {'GOOGL': -11}, {'NVDA': 8}, {'PAAS': -0.7112573326090406}, {'CTVA': -0.6694142433808707}, {'QQQ': 12}, {'NEE': -6}, {'XLK': 10}, {'XLY': -11}, {'TRV': -7}, {'XLF': -0.5999351767783025}, {'MSFT': -14}, {'LRCX': 14}, {'SCCO': 10}, {'XLB': 29}, {'SLV': 31}, {'DD': 25}, {'MAGS': -0.015637583158799373}, {'SOXX': 5}, {'MPLX': -1.0269695405957364}, {'XLU': 32}, {'CI': 8}, {'D': 36}]


In [8]:
import pandas as pd
from IPython.display import display

# 输入的 symbol 列表
index = symbol_list1  # 你可以根据需要修改这个列表

# Convert ma_results to a DataFrame
ma_data = []
for item in ma_results:
    for symbol, ma_values in item.items():
        ma_data.append({'symbol': symbol, 'ma5': ma_values['ma5'], 'ma20': ma_values['ma20']})

ma_df = pd.DataFrame(ma_data)

# Convert kdj_results to a DataFrame
kdj_data = []
for item in kdj_results:
    for symbol, kdj_values in item.items():
        kdj_data.append({'symbol': symbol, 'kd': kdj_values['kd'], 'j': kdj_values['j']})

kdj_df = pd.DataFrame(kdj_data)

# Convert macd_results to a DataFrame
macd_data = []
for item in macd_results:
    for symbol, macd_value in item.items():
        macd_data.append({'symbol': symbol, 'macd': macd_value})

macd_df = pd.DataFrame(macd_data)

# Merge the three DataFrames on the 'symbol' column
merged_df = pd.merge(ma_df, kdj_df, on='symbol')
merged_df = pd.merge(merged_df, macd_df, on='symbol')

# 将 macd 列中的整数显示为整数，非整数保留
def format_macd(value):
    if isinstance(value, float) and value.is_integer():
        return int(value)
    return value

merged_df['macd'] = merged_df['macd'].apply(format_macd)

# 设置 Pandas 的显示格式，去掉多余的小数点
pd.set_option('display.float_format', '{:.2f}'.format)

def calculate_score(row):
    score = 0
    
    # Conditions for ma5
    if row['ma5'] < 0:
        score -= 1
    elif row['ma5'] > 0:
        score += 1
    
    # Conditions for ma20
    if row['ma20'] < 0:
        score -= 1
    elif row['ma20'] > 0:
        score += 1
    
    # Conditions for kd
    if row['kd'] < 0:
        score -= 1
    elif 30 > row['kd'] > 15 or -30 < row['kd'] < -15:
        score += 0
    elif row['kd'] > 0:
        score += 1
    
    # Conditions for j
    if row['j'] > 3:
        score -= 1
    elif row['j'] < -3:
        score += 1
    elif row['j'] == 0:
        score += 0
    else:
        score += 0
    
    # Conditions for macd
    if row['macd'] > 0:
        score += 1
    elif row['macd'] < 0:
        score -= 1
    else:
        score += 0
    
    return score

# 计算 score 列
merged_df['score'] = merged_df.apply(calculate_score, axis=1)

# 根据 score 列排序
merged_df = merged_df.sort_values(by='score', ascending=False)

# 过滤出指定的 symbol
filtered_df = merged_df[merged_df['symbol'].isin(index)]

def highlight_row(row):
    styles = [''] * len(row)
    
    # Conditions for ma5
    if row['ma5'] < 0:
        styles[row.index.get_loc('ma5')] = 'background-color: red; color: white'
    elif row['ma5'] > 0:
        styles[row.index.get_loc('ma5')] = 'background-color: green; color: white'
    
    # Conditions for ma20
    if row['ma20'] < 0:
        styles[row.index.get_loc('ma20')] = 'background-color: red; color: white'
    elif row['ma20'] > 0:
        styles[row.index.get_loc('ma20')] = 'background-color: green; color: white'
    
    # Conditions for kd
    if row['kd'] < 0:
        styles[row.index.get_loc('kd')] = 'background-color: red; color: white'
    elif 30 > row['kd'] > 15 or -30 < row['kd'] < -15:
        styles[row.index.get_loc('kd')] = 'background-color: yellow; color: black'
    elif row['kd'] > 0:
        styles[row.index.get_loc('kd')] = 'background-color: green; color: white'
    
    # Conditions for j
    if row['j'] > 3:
        styles[row.index.get_loc('j')] = 'background-color: red; color: white'
    elif row['j'] < -3:
        styles[row.index.get_loc('j')] = 'background-color: green; color: white'
    elif row['j'] == 0:
        styles[row.index.get_loc('j')] = 'background-color: white; color: black'
    else:
        styles[row.index.get_loc('j')] = 'background-color: yellow'
    
    # Conditions for macd
    if row['macd'] > 0:
        styles[row.index.get_loc('macd')] = 'background-color: green; color: white'
    elif row['macd'] < 0:
        styles[row.index.get_loc('macd')] = 'background-color: red; color: white'
    else:
        styles[row.index.get_loc('macd')] = 'background-color: yellow; color: black'
    
    return styles

# Apply styling
styled_table = filtered_df.style.apply(highlight_row, axis=1)

# Display styled table
display(styled_table)

,symbol,ma5,ma20,kd,j,macd,score
14,GLD,100.000000,100.000000,100.000000,0,33.000000,4
8,XLC,100.000000,100.000000,100.000000,0,21.000000,4
7,SPY,100.000000,100.000000,100.000000,3,23.000000,4
25,XLE,100.000000,100.000000,7.280000,0,3.000000,4
21,XLRE,100.000000,100.000000,100.000000,0,25.000000,4
44,SOXX,100.000000,100.000000,100.000000,0,5.000000,4
33,XLK,100.000000,100.000000,100.000000,0,10.000000,4
46,XLU,100.000000,100.000000,100.000000,0,32.000000,4
20,XLP,100.000000,100.000000,100.000000,1,22.000000,4
31,QQQ,100.000000,100.000000,100.000000,5,12.000000,3


In [9]:
import pandas as pd
from IPython.display import display

# 输入的 symbol 列表
index = symbol_list2  # 你可以根据需要修改这个列表

# Convert ma_results to a DataFrame
ma_data = []
for item in ma_results:
    for symbol, ma_values in item.items():
        ma_data.append({'symbol': symbol, 'ma5': ma_values['ma5'], 'ma20': ma_values['ma20']})

ma_df = pd.DataFrame(ma_data)

# Convert kdj_results to a DataFrame
kdj_data = []
for item in kdj_results:
    for symbol, kdj_values in item.items():
        kdj_data.append({'symbol': symbol, 'kd': kdj_values['kd'], 'j': kdj_values['j']})

kdj_df = pd.DataFrame(kdj_data)

# Convert macd_results to a DataFrame
macd_data = []
for item in macd_results:
    for symbol, macd_value in item.items():
        macd_data.append({'symbol': symbol, 'macd': macd_value})

macd_df = pd.DataFrame(macd_data)

# Merge the three DataFrames on the 'symbol' column
merged_df = pd.merge(ma_df, kdj_df, on='symbol')
merged_df = pd.merge(merged_df, macd_df, on='symbol')

# 将 macd 列中的整数显示为整数，非整数保留
def format_macd(value):
    if isinstance(value, float) and value.is_integer():
        return int(value)
    return value

merged_df['macd'] = merged_df['macd'].apply(format_macd)

# 设置 Pandas 的显示格式，去掉多余的小数点
pd.set_option('display.float_format', '{:.2f}'.format)

def calculate_score(row):
    score = 0
    
    # Conditions for ma5
    if row['ma5'] < 0:
        score -= 1
    elif row['ma5'] > 0:
        score += 1
    
    # Conditions for ma20
    if row['ma20'] < 0:
        score -= 1
    elif row['ma20'] > 0:
        score += 1
    
    # Conditions for kd
    if row['kd'] < 0:
        score -= 1
    elif 30 > row['kd'] > 15 or -30 < row['kd'] < -15:
        score += 0
    elif row['kd'] > 0:
        score += 1
    
    # Conditions for j
    if row['j'] > 3:
        score -= 1
    elif row['j'] < -3:
        score += 1
    elif row['j'] == 0:
        score += 0
    else:
        score += 0
    
    # Conditions for macd
    if row['macd'] > 0:
        score += 1
    elif row['macd'] < 0:
        score -= 1
    else:
        score += 0
    
    return score

# 计算 score 列
merged_df['score'] = merged_df.apply(calculate_score, axis=1)

# 根据 score 列排序
merged_df = merged_df.sort_values(by='score', ascending=False)

# 过滤出指定的 symbol
filtered_df = merged_df[merged_df['symbol'].isin(index)]

def highlight_row(row):
    styles = [''] * len(row)
    
    # Conditions for ma5
    if row['ma5'] < 0:
        styles[row.index.get_loc('ma5')] = 'background-color: red; color: white'
    elif row['ma5'] > 0:
        styles[row.index.get_loc('ma5')] = 'background-color: green; color: white'
    
    # Conditions for ma20
    if row['ma20'] < 0:
        styles[row.index.get_loc('ma20')] = 'background-color: red; color: white'
    elif row['ma20'] > 0:
        styles[row.index.get_loc('ma20')] = 'background-color: green; color: white'
    
    # Conditions for kd
    if row['kd'] < 0:
        styles[row.index.get_loc('kd')] = 'background-color: red; color: white'
    elif 30 > row['kd'] > 15 or -30 < row['kd'] < -15:
        styles[row.index.get_loc('kd')] = 'background-color: yellow; color: black'
    elif row['kd'] > 0:
        styles[row.index.get_loc('kd')] = 'background-color: green; color: white'
    
    # Conditions for j
    if row['j'] > 3:
        styles[row.index.get_loc('j')] = 'background-color: red; color: white'
    elif row['j'] < -3:
        styles[row.index.get_loc('j')] = 'background-color: green; color: white'
    elif row['j'] == 0:
        styles[row.index.get_loc('j')] = 'background-color: white; color: black'
    else:
        styles[row.index.get_loc('j')] = 'background-color: yellow'
    
    # Conditions for macd
    if row['macd'] > 0:
        styles[row.index.get_loc('macd')] = 'background-color: green; color: white'
    elif row['macd'] < 0:
        styles[row.index.get_loc('macd')] = 'background-color: red; color: white'
    else:
        styles[row.index.get_loc('macd')] = 'background-color: yellow; color: black'
    
    return styles

# Apply styling
styled_table = filtered_df.style.apply(highlight_row, axis=1)

# Display styled table
display(styled_table)

,symbol,ma5,ma20,kd,j,macd,score
4,UBER,100.000000,100.000000,100.000000,0,10.000000,4
6,HAS,100.000000,100.000000,100.000000,0,30.000000,4
24,NEM,100.000000,100.000000,100.000000,0,33.000000,4
47,CI,100.000000,100.000000,1.950000,0,8.000000,4
28,NVDA,100.000000,100.000000,100.000000,0,8.000000,4
1,SVM,100.000000,100.000000,-0.630000,0,48.000000,2
3,BKR,100.000000,100.000000,1.700000,0,-0.753234,2
38,LRCX,100.000000,100.000000,-1.410000,0,14.000000,2
15,AMD,0.858294,0.052779,14.260000,0,-2.348453,2
39,SCCO,-0.700598,1.955972,-1.570000,0,10.000000,0


In [10]:
import pandas as pd
from IPython.display import display

# 输入的 symbol 列表
index = symbol_list3  # 你可以根据需要修改这个列表

# Convert ma_results to a DataFrame
ma_data = []
for item in ma_results:
    for symbol, ma_values in item.items():
        ma_data.append({'symbol': symbol, 'ma5': ma_values['ma5'], 'ma20': ma_values['ma20']})

ma_df = pd.DataFrame(ma_data)

# Convert kdj_results to a DataFrame
kdj_data = []
for item in kdj_results:
    for symbol, kdj_values in item.items():
        kdj_data.append({'symbol': symbol, 'kd': kdj_values['kd'], 'j': kdj_values['j']})

kdj_df = pd.DataFrame(kdj_data)

# Convert macd_results to a DataFrame
macd_data = []
for item in macd_results:
    for symbol, macd_value in item.items():
        macd_data.append({'symbol': symbol, 'macd': macd_value})

macd_df = pd.DataFrame(macd_data)

# Merge the three DataFrames on the 'symbol' column
merged_df = pd.merge(ma_df, kdj_df, on='symbol')
merged_df = pd.merge(merged_df, macd_df, on='symbol')

# 将 macd 列中的整数显示为整数，非整数保留
def format_macd(value):
    if isinstance(value, float) and value.is_integer():
        return int(value)
    return value

merged_df['macd'] = merged_df['macd'].apply(format_macd)

# 设置 Pandas 的显示格式，去掉多余的小数点
pd.set_option('display.float_format', '{:.2f}'.format)

def calculate_score(row):
    score = 0
    
    # Conditions for ma5
    if row['ma5'] < 0:
        score -= 1
    elif row['ma5'] > 0:
        score += 1
    
    # Conditions for ma20
    if row['ma20'] < 0:
        score -= 1
    elif row['ma20'] > 0:
        score += 1
    
    # Conditions for kd
    if row['kd'] < 0:
        score -= 1
    elif 30 > row['kd'] > 15 or -30 < row['kd'] < -15:
        score += 0
    elif row['kd'] > 0:
        score += 1
    
    # Conditions for j
    if row['j'] > 3:
        score -= 1
    elif row['j'] < -3:
        score += 1
    elif row['j'] == 0:
        score += 0
    else:
        score += 0
    
    # Conditions for macd
    if row['macd'] > 0:
        score += 1
    elif row['macd'] < 0:
        score -= 1
    else:
        score += 0
    
    return score

# 计算 score 列
merged_df['score'] = merged_df.apply(calculate_score, axis=1)

# 根据 score 列排序
merged_df = merged_df.sort_values(by='score', ascending=False)

# 过滤出指定的 symbol
filtered_df = merged_df[merged_df['symbol'].isin(index)]

def highlight_row(row):
    styles = [''] * len(row)
    
    # Conditions for ma5
    if row['ma5'] < 0:
        styles[row.index.get_loc('ma5')] = 'background-color: red; color: white'
    elif row['ma5'] > 0:
        styles[row.index.get_loc('ma5')] = 'background-color: green; color: white'
    
    # Conditions for ma20
    if row['ma20'] < 0:
        styles[row.index.get_loc('ma20')] = 'background-color: red; color: white'
    elif row['ma20'] > 0:
        styles[row.index.get_loc('ma20')] = 'background-color: green; color: white'
    
    # Conditions for kd
    if row['kd'] < 0:
        styles[row.index.get_loc('kd')] = 'background-color: red; color: white'
    elif 30 > row['kd'] > 15 or -30 < row['kd'] < -15:
        styles[row.index.get_loc('kd')] = 'background-color: yellow; color: black'
    elif row['kd'] > 0:
        styles[row.index.get_loc('kd')] = 'background-color: green; color: white'
    
    # Conditions for j
    if row['j'] > 3:
        styles[row.index.get_loc('j')] = 'background-color: red; color: white'
    elif row['j'] < -3:
        styles[row.index.get_loc('j')] = 'background-color: green; color: white'
    elif row['j'] == 0:
        styles[row.index.get_loc('j')] = 'background-color: white; color: black'
    else:
        styles[row.index.get_loc('j')] = 'background-color: yellow'
    
    # Conditions for macd
    if row['macd'] > 0:
        styles[row.index.get_loc('macd')] = 'background-color: green; color: white'
    elif row['macd'] < 0:
        styles[row.index.get_loc('macd')] = 'background-color: red; color: white'
    else:
        styles[row.index.get_loc('macd')] = 'background-color: yellow; color: black'
    
    return styles

# Apply styling
styled_table = filtered_df.style.apply(highlight_row, axis=1)

# Display styled table
display(styled_table)

,symbol,ma5,ma20,kd,j,macd,score
28,NVDA,100.000000,100.000000,100.000000,0,8.000000,4
23,AAPL,100.000000,100.000000,11.950000,4,16.000000,3
37,MSFT,0.601754,-1.408585,12.520000,0,-14.000000,0
11,TSLA,0.883599,-3.862588,12.930000,0,-39.000000,0
27,GOOGL,0.072379,-3.569495,4.430000,0,-11.000000,0
26,META,-1.958408,0.858654,-21.050000,0,-23.087700,-2
17,AMZN,-100.000000,-100.000000,-0.400000,0,-0.082623,-4
